This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-features/parallel-inference-aloha-tutorial).

## Aloha Parallel Inference Demonstration

This tutorial will focus on the Pipeline method `parallel_infer`, which allows a List of data to be submitted to a Wallaroo instance for parallel inference requests.  This provides high speed increases in situations where data has to be broken up for size and memory needs, data is requested from multiple sources and submitted in a single request, or other use cases.

For this example we will be using an open source model that uses an [Aloha CNN LSTM model](https://www.researchgate.net/publication/348920204_Using_Auxiliary_Inputs_in_Deep_Learning_Models_for_Detecting_DGA-based_Domain_Names) for classifying Domain names as being either legitimate or being used for nefarious purposes such as malware distribution.  

## Tutorial Goals

* Create a workspace for our work.
* Upload the Aloha TensorFlow model.
* Create a pipeline that can ingest our submitted data, submit it to the model, and export the results.
* Run a sample inference through our pipeline by loading a file.
* Run a batch inference to show submitting a set of data to an inference request.
* Split a DataFrame into a List of 1,000 separate DataFrames to simulate separate inference requests.
* Submit the List of DataFrames sequentially and display how long this takes.
* Submit the same List of DataFrames with `parallel_infer` and compare how long it takes.

## Prerequisites

* A Wallaroo version 2023.2.1 and above instance.

## Reference

[Wallaroo SDK Essentials Guide: Inference Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-inferences/#parallel-inferences)

## Open a Connection to Wallaroo

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

In [1]:
import wallaroo
import asyncio 
import datetime
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)

# to display dataframe tables
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Client connection from local Wallaroo instance

wl = wallaroo.Client(auth_type="sso")

Please log into the following URL in a web browser:

	https://keycloak.autoscale-uat-gcp.wallaroo.dev/auth/realms/master/device?user_code=PZDK-RRVR

Login successful!


## Create the Workspace

We will create a workspace to work in and call it the "alohaworkspace", then set it as current workspace environment.  We'll also create our pipeline in advance as `alohapipeline`.  The model name and the model file will be specified for use in later steps.

To allow this tutorial to be run multiple times or by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.

In [3]:
import string
import random

# make a random 4 character prefix
prefix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))
workspace_name = f"{prefix}alohaworkspace"
pipeline_name = f"{prefix}alohapipeline"
model_name = f"{prefix}alohamodel"
model_file_name = './models/alohacnnlstm.zip'

In [4]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [5]:
wl.list_workspaces()

Name,Created At,Users,Models,Pipelines
younes.amar@wallaroo.ai - Default Workspace,2024-02-22 19:00:03,"['younes.amar@wallaroo.ai', 'jeff.will@wallaroo.ai']",2,3
jeff.will@wallaroo.ai - Default Workspace,2024-02-22 22:38:59,['jeff.will@wallaroo.ai'],1,1
house-price-prediction,2024-02-23 14:58:29,"['younes.amar@wallaroo.ai', 'john.riley@wallaroo.ai', 'jeff.will@wallaroo.ai']",1,1


In [7]:
##aloha_pipeline = wl.list_pipelines()[1]
##workspace = wl.list_workspaces()[2]

In [6]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

aloha_pipeline = get_pipeline(pipeline_name)
aloha_pipeline

name,icemalohapipeline
created,2024-02-27 18:11:13.931619+00:00
last_updated,2024-02-27 18:11:13.931619+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,4ed48597-edfa-42c3-a79b-18d84e9112e5
steps,
published,False


# Upload the Models

Now we will upload our models.  Note that for this example we are applying the model from a .ZIP file.  The Aloha model is a [protobuf](https://developers.google.com/protocol-buffers) file that has been defined for evaluating web pages, and we will configure it to use data in the `tensorflow` format.

In [7]:
model = wl.upload_model(model_name, model_file_name, framework=wallaroo.framework.Framework.TENSORFLOW).configure("tensorflow")

## Deploy a model

Now that we have a model that we want to use we will create a deployment for it. 

We will tell the deployment we are using a tensorflow model and give the deployment name and the configuration we want for the deployment.

In [8]:
aloha_pipeline.add_model_step(model)

name,icemalohapipeline
created,2024-02-27 18:11:13.931619+00:00
last_updated,2024-02-27 18:11:13.931619+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,4ed48597-edfa-42c3-a79b-18d84e9112e5
steps,
published,False


In [9]:
REPLICAS = 8
deployment_config = (wallaroo.DeploymentConfigBuilder()
    .replica_count(REPLICAS)
    .cpus(0.25)
    .memory("1Gi")
    .build())

In [10]:
aloha_pipeline = aloha_pipeline.deploy(deployment_config =deployment_config)

Waiting for deployment - this will take up to 45s ......................... ok


We can verify that the pipeline is running and list what models are associated with it.

In [11]:
aloha_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.124.1.50',
   'name': 'engine-847dc8654b-cfzsg',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'icemalohapipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'config': {'batch_config': None,
       'filter_threshold': None,
       'id': 18,
       'input_schema': None,
       'model_version_id': 13,
       'output_schema': None,
       'runtime': 'tensorflow',
       'sidekick_uri': None,
       'tensor_fields': None},
      'model_version': {'conversion': {'framework': 'tensorflow',
        'python_version': '3.8',
        'requirements': []},
       'file_info': {'file_name': 'alohacnnlstm.zip',
        'sha': 'd71d9ffc61aaac58c2b1ed70a2db13d1416fb9d3f5b891e5e4e2e97180fe22f8',
        'version': 'a2839dbb-f9fd-45ff-ae4e-c11b0e8ceb1e'},
       'id': 13,
       'image_path': None,
       'name': 'icemalohamodel',
       'status': 'ready',
    

## Interferences

### Infer 1 row

Now that the pipeline is deployed and our Aloha model is in place, we'll perform a smoke test to verify the pipeline is up and running properly.  We'll use the `infer_from_file` command to load a single encoded URL into the inference engine and print the results back out.

The result should tell us that the tokenized URL is legitimate (0) or fraud (1).  This sample data should return close to 0.

In [29]:
result = aloha_pipeline.infer_from_file('./data/data_1.df.json')

display(result)

,time,in.text_input,out.banjori,out.corebot,out.cryptolocker,out.dircrypt,out.gozi,out.kraken,out.locky,out.main,out.matsnu,out.pykspa,out.qakbot,out.ramdo,out.ramnit,out.simda,out.suppobox,anomaly.count
0,2024-02-27 21:56:56.484,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]",[0.0015195842],[0.98291475],[0.012099553],[4.7591206e-05],[2.0289332e-05],[0.00031977228],[0.011029261],[0.997564],[0.010341614],[0.008038961],[0.016155045],[0.0062362333],[0.0009985747],[1.7933435e-26],[1.3889844e-27],0


### Batch Inference Example

Now we'll perform a batch inference.  We have the file `./data/data_25k.df.json`, which is a pandas DataFrame file with 25,000 records to analyze.  We'll provide it to the pipeline and perform a sample inference, and provide the first 20 rows.

In [17]:
%time

test_data = pd.read_json("./data/data_25k.df.json")


batch_result = aloha_pipeline.infer(test_data.head(1000))
display(batch_result.head(20))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


,time,in.text_input,out.banjori,out.corebot,out.cryptolocker,out.dircrypt,out.gozi,out.kraken,out.locky,out.main,out.matsnu,out.pykspa,out.qakbot,out.ramdo,out.ramnit,out.simda,out.suppobox,anomaly.count
0,2024-02-27 20:03:33.983,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]",[0.0015195814],[0.98291475],[0.012099549],[4.7591115e-05],[2.0289312e-05],[0.00031977257],[0.011029262],[0.997564],[0.010341609],[0.008038961],[0.016155047],[0.00623623],[0.0009985747],[1.7933434e-26],[1.388995e-27],0
1,2024-02-27 20:03:33.983,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 20, 19, 27, 14, 17, 24, 13, 23, 20, 18, 35, 18, 22, 23]",[7.447168e-18],[6.7359245e-08],[0.17081994],[1.3220122e-09],[1.2758657e-24],[0.22559547],[0.3420985],[0.99999994],[0.3080186],[0.1828217],[3.8022408e-11],[0.2062254],[0.15215829],[1.1701982e-30],[3.1513975e-38],0
2,2024-02-27 20:03:33.983,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 33, 25, 36, 25, 31, 14, 32, 36, 25, 12, 35, 34, 30, 28, 27, 24, 29, 27]",[2.8598758e-21],[9.302004e-08],[0.044453003],[6.16377e-09],[8.3496755e-23],[0.48234475],[0.26332903],[1.0],[0.29800338],[0.22361776],[1.5238921e-06],[0.32820404],[0.029332504],[1.1995622e-31],[0.0],0
3,2024-02-27 20:03:33.983,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 23, 22, 15, 12, 35, 34, 36, 12, 18, 24, 34, 32, 36, 12, 14, 16, 27, 22, 23]",[2.138713e-15],[3.881756e-10],[0.045599725],[1.9090368e-07],[1.3140173e-25],[0.5954264],[0.17374137],[0.9999997],[0.23151578],[0.1759168],[1.0876152e-09],[0.2183228],[0.012869265],[6.1588803e-28],[1.4386127e-35],0
4,2024-02-27 20:03:33.983,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 13, 14, 12, 33, 16, 23, 15, 22, 30, 28, 26, 12, 16, 32, 37, 29, 22, 28, 22, 16, 27, 32]",[9.453379e-15],[7.091178e-10],[0.049815144],[5.2914135e-09],[7.4132087e-19],[1.5504633e-13],[1.0791851e-15],[0.9999989],[1.5003075e-15],[0.3307571],[2.62589e-07],[0.5036279],[0.020393759],[0.0],[2.3292148e-38],0
5,2024-02-27 20:03:33.983,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 29, 20, 33, 13, 36, 35, 30, 21, 29, 17, 26, 19, 25, 36, 14, 23, 16, 18, 15, 21, 18, 28, 35, 19]",[1.7247285e-17],[8.1354045e-08],[0.013697123],[5.608618e-11],[1.4032912e-17],[0.49469122],[0.1197886],[0.99999994],[0.19000013],[0.105966926],[5.524429e-06],[0.24210057],[0.0069435053],[1.2804911e-34],[9.482465e-35],0
6,2024-02-27 20:03:33.983,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22, 36, 14, 12, 23, 14, 13, 20, 20, 23, 27, 36, 29, 35, 19, 33, 22, 25, 26, 32, 21]",[5.5500796e-18],[3.360874e-07],[0.023452949],[1.1318812e-10],[1.0496917e-22],[0.23692918],[0.06445695],[0.99999183],[0.07306594],[0.06499429],[1.4302739e-08],[0.11925242],[0.0011031039],[1.520634e-32],[0.0],0
7,2024-02-27 20:03:33.983,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36, 22, 28, 23, 20, 25, 21, 20, 16, 12, 33, 21, 14, 34, 34, 32, 19, 36, 17, 29, 26, 14, 29]",[3.9222717e-18],[1.4074378e-10],[0.0109469],[8.202828e-11],[2.4549838e-24],[0.42107272],[0.071240015],[0.9982491],[0.118182994],[0.08340969],[1.9207924e-09],[0.16958168],[0.0005199056],[0.0],[0.0],0
8,2024-02-27 20:03:33.983,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 30, 33, 29, 37, 24, 33, 16, 20, 24]",[4.0574582e-11],[1.0878912e-09],[0.17916855],[1.7313005e-06],[8.697294e-18],[9.197087e-16],[3.8521368e-17],[0.9999977],[3.2654394e-17],[0.32568422],[6.834302e-09],[0.3700782],[0.44918332],[0.0],[2.0823951e-26],0
9,2024-02-27 20:03:33.983,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### Parallel Inference Example

This time, let's take the same file and split it into 1,000 **separate** DataFrames, which each indivual row as a single DataFrame.  This is toy data; we're just providing it as an example of how to submit a an inference request for parallel infer.

In [19]:
test_data = pd.read_json("./data/data_25k.df.json")
test_list = []

for index, row in test_data.head(1000).iterrows():
    test_list.append(row.to_frame('text_input').reset_index())


Now we'll perform an inference with Parallel Infer through the pipeline.

The pipeline `parallel_infer(tensor_list, timeout, num_parallel, retries)` **asynchronous** method performs an inference as defined by the pipeline steps and takes the following arguments:

* **tensor_list** (*REQUIRED List*): The data submitted to the pipeline for inference as a List of the supported data types:
  * [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html):  Data submitted as a pandas DataFrame are returned as a pandas DataFrame.  For models that output one column  based on the models outputs.
  * [Apache Arrow](https://arrow.apache.org/) (**Preferred**): Data submitted as an Apache Arrow are returned as an Apache Arrow.
* **timeout** (*OPTIONAL int*): A timeout in seconds before the inference throws an exception.  The default is 15 second per call to accommodate large, complex models.  Note that for a batch inference, this is **per list item** - with 10 inference requests, each would have a default timeout of 15 seconds.
* **num_parallel** (*OPTIONAL int*):  The number of parallel threads used for the submission.  **This should be no more than four times the number of pipeline replicas**.
* **retries** (*OPTIONAL int*):  The number of retries per inference request submitted.

`parallel_infer` is an asynchronous method that returns the Python callback list of tasks. Calling `parallel_infer` should be called with the `await` keyword to retrieve the callback results.

First we'll process the 1,000 rows serially and clock how long this takes.  This may take up to 3-10 minutes depending on the speed of the connection between the client and the Wallaroo instance.

In [32]:
#
# Run the inference sequentially to establish a baseline
#
now = datetime.datetime.now()

#test_data_df = pd.read_json("./data/data_25k.df.json")
aloha_pipeline.infer_from_file("./data/data_25k.df.json", timeout=1200)

total_sequential = datetime.datetime.now() - now

print(f"Elapsed = {total_sequential.total_seconds()} : {len(results)}")

RuntimeError: Inference did not return within 1200s, adjust if necessary

Now we'll compare that to using the `parallel_infer` method.  The same data, but now submitted as multiple rows of the list of DataFrames at one time.

In [23]:
test_data_df = pd.read_json("./data/data_25k.df.json")
display(test_data_df)

,text_input
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]"
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 20, 19, 27, 14, 17, 24, 13, 23, 20, 18, 35, 18, 22, 23]"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 33, 25, 36, 25, 31, 14, 32, 36, 25, 12, 35, 34, 30, 28, 27, 24, 29, 27]"
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 23, 22, 15, 12, 35, 34, 36, 12, 18, 24, 34, 32, 36, 12, 14, 16, 27, 22, 23]"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 13, 14, 12, 33, 16, 23, 15, 22, 30, 28, 26, 12, 16, 32, 37, 29, 22, 28, 22, 16, 27, 32]"
...,...
24949,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 13, 21, 26, 21, 24, 19, 15, 19, 36, 30, 31, 25, 30, 29]"
24950,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 14, 12, 13, 24, 18, 19, 20, 26, 12, 25, 12, 33, 26]"
24951,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 31, 34, 12, 19, 16, 18, 25, 17, 26, 26, 18, 27, 27]"
24952,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 34, 21, 13, 12, 22, 26, 26, 21, 17, 27, 28, 19, 33, 23]"


## Parallel infer with dataframe and arrow table

In [ ]:
timeout_secs=1200
now = datetime.datetime.now()
##########
parallel_results = await aloha_pipeline.parallel_infer(tensor=test_data_df, 
                                                       timeout=timeout_secs, 
                                                       num_parallel=2*REPLICAS, 
                                                       retries=3)
##########
total_parallel = datetime.datetime.now() - now
print(f"Elapsed_in_parallel = {total_parallel.total_seconds()} : {len(parallel_results)}")

Elapsed_in_parallel = 284.77102 : 24954


In [21]:
test_data_df = pd.read_json("./data/data_5-error.df.json")

In [26]:
now = datetime.datetime.now()
##########
parallel_results = await aloha_pipeline.parallel_infer(tensor=test_data_df, 
                                                       timeout=timeout_secs, 
                                                       num_parallel=2*REPLICAS)
##########
total_parallel = datetime.datetime.now() - now
print(f"Elapsed_in_parallel = {total_parallel.total_seconds()} : {len(parallel_results)}")

Elapsed_in_parallel = 287.071682 : 24954


In [27]:
parallel_results

,time,in.text_input,out.banjori,out.corebot,out.cryptolocker,out.dircrypt,out.gozi,out.kraken,out.locky,out.main,out.matsnu,out.pykspa,out.qakbot,out.ramdo,out.ramnit,out.simda,out.suppobox,anomaly.count,error
0,2024-02-26 20:04:29.418000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]",[0.0015195857],[0.9829148],[0.012099565],[4.7591297e-05],[2.0289372e-05],[0.0003197726],[0.011029283],[0.997564],[0.010341615],[0.008038961],[0.016155062],[0.006236233],[0.0009985751],[1.793378e-26],[1.3889951e-27],0,
1,,,,,,,,,,,,,,,,,,,Server error '503 Service Unavailable' for url 'http://engine-lb.phrgalohapipeline-2:29502/pipelines/phrgalohapipeline?dataset%5B%5D=%2A&dataset.exclude%5B%5D=metadata&dataset.separator=.'\nFor more information check: https://httpstatuses.com/503
2,,,,,,,,,,,,,,,,,,,Server error '503 Service Unavailable' for url 'http://engine-lb.phrgalohapipeline-2:29502/pipelines/phrgalohapipeline?dataset%5B%5D=%2A&dataset.exclude%5B%5D=metadata&dataset.separator=.'\nFor more information check: https://httpstatuses.com/503
3,2024-02-26 20:04:29.419000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]",[0.0015195857],[0.9829148],[0.012099565],[4.7591297e-05],[2.0289372e-05],[0.0003197726],[0.011029283],[0.997564],[0.010341615],[0.008038961],[0.016155062],[0.006236233],[0.0009985751],[1.793378e-26],[1.3889951e-27],0,
4,2024-02-26 20:04:29.419000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]",[0.0015195857],[0.9829148],[0.012099565],[4.7591297e-05],[2.0289372e-05],[0.0003197726],[0.011029283],[0.997564],[0.010341615],[0.008038961],[0.016155062],[0.006236233],[0.0009985751],[1.793378e-26],[1.3889951e-27],0,


In [ ]:
print(f"Comparison:\nTotal Time Sequentially: {total_sequential.total_seconds()}\nTotal Time Paralleled: {total_parallel.total_seconds()}")

Depending on the connection and other requirements, the differences in time can be immense.  For a local connection, the time to process the List sequentially took 4 minutes - versus 13 seconds for the `parallel_infer` method.  This is an immense difference.

In [28]:
import pyarrow as pa

In [29]:
with pa.ipc.open_file("data/data_1k.arrow") as reader:
    test_table = reader.read_all()

In [30]:
test_table[0:2]

pyarrow.Table
text_input: fixed_size_list<item: float>[50]
  child 0, item: float
----
text_input: [[[0,0,0,0,0,...,32,30,19,26,17],[0,0,0,0,0,...,29,12,36,31,12]]]

In [37]:
data1_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,16,32,23,29,32,30,19,26,17]
data2_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,16,32,23,29,32,30,19,26,17]
data3_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,16,32,23,29,32,30,19,26,19000]

In [38]:
new_test_table1 = pa.table({"text_input": [data1_list, data2_list, data3_list]}, schema=test_table.schema)
new_test_table1

pyarrow.Table
text_input: fixed_size_list<item: float>[50]
  child 0, item: float
----
text_input: [[[0,0,0,0,0,...,32,30,19,26,17],[0,0,0,0,0,...,32,30,19,26,17],[0,0,0,0,0,...,32,30,19,26,19000]]]

In [39]:
new_test_table1

pyarrow.Table
text_input: fixed_size_list<item: float>[50]
  child 0, item: float
----
text_input: [[[0,0,0,0,0,...,32,30,19,26,17],[0,0,0,0,0,...,32,30,19,26,17],[0,0,0,0,0,...,32,30,19,26,19000]]]

In [40]:
timeout_secs=1200
now = datetime.datetime.now()
##########
parallel_results = await aloha_pipeline.parallel_infer(tensor=new_test_table1, 
                                                       timeout=timeout_secs, 
                                                       num_parallel=2*REPLICAS, 
                                                       retries=3)
##########
total_parallel = datetime.datetime.now() - now
print(f"Elapsed_in_parallel = {total_parallel.total_seconds()} : {len(parallel_results)}")

Elapsed_in_parallel = 1.072185 : 3


In [41]:
parallel_results["error"]

[
  [
    "",
    "",
    "Server error '500 Internal Server Error' for url 'http://engine-lb.phrgalohapipeline-2:29502/pipelines/phrgalohapipeline?dataset%5B%5D=%2A&dataset.exclude%5B%5D=metadata&dataset.separator=.'
For more information check: https://httpstatuses.com/500"
  ]
]

## Undeploy Pipeline

When finished with our tests, we will undeploy the pipeline so we have the Kubernetes resources back for other tasks.  Note that if the deployment variable is unchanged aloha_pipeline.deploy() will restart the inference engine in the same configuration as before.

In [ ]:
aloha_pipeline.undeploy()